# Implementação do Simplex

### Função `inicializar_tabela`
Esta função cria a tabela inicial do Simplex. Ela adiciona variáveis de folga às restrições e organiza os coeficientes da função objetivo e das restrições em uma matriz. A tabela será usada como base para o algoritmo.

## Fórmula da Função Objetivo

$$
Z = c_1x_1 + c_2x_2 + \dots + c_nx_n
$$

In [14]:
def inicializar_tabela(c, A, b):
    """
    Inicializa a tabela do Simplex.
    c: Coeficientes da função objetivo (lista).
    A: Matriz de restrições (lista de listas).
    b: Lado direito das restrições (lista).
    Retorna a tabela inicial.
    """
    # Adiciona as variáveis de folga
    num_restricoes = len(A)
    num_variaveis = len(c)
    tabela = []

    for i in range(num_restricoes):
        linha = A[i] + [0] * num_restricoes + [b[i]]
        linha[num_variaveis + i] = 1  # Variável de folga
        tabela.append(linha)

    # Adiciona a linha da função objetivo
    linha_objetivo = c + [0] * (num_restricoes + 1)
    tabela.append(linha_objetivo)

    return tabela

### Função `encontrar_coluna_pivo`
Esta função identifica a **coluna pivô**, que corresponde à variável que entra na base. A coluna pivô é escolhida com base no coeficiente mais negativo na linha da função objetivo, indicando a variável que mais aumenta o valor da função objetivo.

In [15]:
def encontrar_coluna_pivo(tabela):
    """
    Encontra a coluna pivô (variável de entrada).
    Retorna o índice da coluna.
    """
    linha_objetivo = tabela[-1]
    return linha_objetivo.index(min(linha_objetivo[:-1]))


### Função `encontrar_linha_pivo`
Esta função identifica a **linha pivô**, que corresponde à variável que sai da base. A linha pivô é escolhida com base na menor razão entre o lado direito (`b`) e o coeficiente da coluna pivô, garantindo que a solução permaneça viável.

In [16]:
def encontrar_linha_pivo(tabela, coluna_pivo):
    """
    Encontra a linha pivô (variável de saída).
    Retorna o índice da linha.
    """
    num_linhas = len(tabela) - 1
    menores_razões = []

    for i in range(num_linhas):
        if tabela[i][coluna_pivo] > 0:
            razao = tabela[i][-1] / tabela[i][coluna_pivo]
            menores_razões.append((razao, i))

    if not menores_razões:
        raise ValueError("Problema ilimitado.")

    _, linha_pivo = min(menores_razões)
    return linha_pivo


### Função `realizar_pivoteamento`
Esta função realiza o **pivoteamento**, atualizando a tabela para refletir a troca de variáveis na base. O pivoteamento normaliza a linha pivô e ajusta as outras linhas para zerar os valores na coluna pivô, exceto na linha pivô.

In [17]:
def realizar_pivoteamento(tabela, linha_pivo, coluna_pivo):
    """
    Realiza o pivoteamento na tabela.
    """
    # Normaliza a linha pivô
    fator_pivo = tabela[linha_pivo][coluna_pivo]
    tabela[linha_pivo] = [x / fator_pivo for x in tabela[linha_pivo]]

    # Atualiza as outras linhas
    for i in range(len(tabela)):
        if i != linha_pivo:
            fator = tabela[i][coluna_pivo]
            tabela[i] = [tabela[i][j] - fator * tabela[linha_pivo][j] for j in range(len(tabela[0]))]


### Função `simplex`
Esta é a função principal que implementa o método Simplex. Ela coordena as etapas do algoritmo:
1. Inicializa a tabela.
2. Identifica a coluna e a linha pivô.
3. Realiza o pivoteamento.
4. Calcula a solução ótima e o valor da função objetivo.

In [18]:
def simplex(c, A, b):
    """
    Resolve um problema de programação linear usando o método Simplex.
    c: Coeficientes da função objetivo (lista).
    A: Matriz de restrições (lista de listas).
    b: Lado direito das restrições (lista).
    Retorna a solução ótima e o valor da função objetivo.
    """
    tabela = inicializar_tabela(c, A, b)

    while any(x < 0 for x in tabela[-1][:-1]):
        coluna_pivo = encontrar_coluna_pivo(tabela)
        linha_pivo = encontrar_linha_pivo(tabela, coluna_pivo)
        realizar_pivoteamento(tabela, linha_pivo, coluna_pivo)

    # Solução ótima
    num_variaveis = len(c)
    solucao = [0] * num_variaveis
    for i in range(len(tabela) - 1):
        for j in range(num_variaveis):
            if tabela[i][j] == 1 and all(tabela[k][j] == 0 for k in range(len(tabela)) if k != i):
                solucao[j] = tabela[i][-1]

    valor_otimo = tabela[-1][-1]
    return solucao, valor_otimo



### Resultado
A solução ótima encontrada pelo método Simplex é:
- `x = 2`
- `y = 6`

O valor máximo da função objetivo é `Z = 36`.

In [19]:
# Exemplo de uso
c = [-3, -5]  # Maximizar 3x + 5y
A = [
    [1, 0],
    [0, 2],
    [3, 2]
]
b = [4, 12, 18]

solucao, valor_otimo = simplex(c, A, b)
print("Solução ótima:", solucao)
print("Valor ótimo:", valor_otimo)

Solução ótima: [2.0, 6.0]
Valor ótimo: 36.0
